In [1]:
import torch
import torch.nn.functional as F
from torchvision import transforms  # 是一个常用的图片变换类
from torchvision import datasets
from torch.utils.data import DataLoader
import prettytable

In [2]:
Side_length=8
my_batch_size = Side_length*Side_length
MyTransform = transforms.Compose(
    [
        transforms.ToTensor(),  # 把数据转换成张量
        transforms.Normalize((0.1307,), (0.3081,))  # 0.1307是均值，0.3081是标准差，具体如何计算得到，之后再看
    ]
)
train_dataset = datasets.MNIST(root='./dataset/mnist',train=True,download=False, transform=MyTransform)  # #第一次使用需要设置为true，下载数据集,使用的图片变换类transforms
train_process_data = DataLoader(train_dataset,  shuffle=True, batch_size=my_batch_size)   #shuffle：是否将数据打乱   #batch_size设置为my_batch_size
test_dataset = datasets.MNIST(root='./dataset/mnist',train=False,download=False,transform=MyTransform)
test_process_data = DataLoader(test_dataset,shuffle=True,batch_size=my_batch_size)

In [3]:
class MyCNN_NET(torch.nn.Module):
    def __init__(self):
        super(MyCNN_NET, self).__init__()  #初始化函数为空

        self.L1_conv_pool = torch.nn.Sequential(  # torch.nn.Sequential理解为向网络中增加结构
            torch.nn.Conv2d(1, 26, kernel_size=3), #卷积层，输入通道数目，输出通道数目，卷积核大小为3*3
            torch.nn.BatchNorm2d(26),   #输入batch的每一个特征通道进行normalize，对26个通道进行normalize标准化
            torch.nn.ReLU(inplace=True),  #激活函数relu  inplace为True，将会改变输入的数据 
            torch.nn.MaxPool2d(kernel_size=2, stride=2)  
            #kernel_size ：表示做最大池化的窗口大小，可以是单个值，也可以是tuple元组
            #stride ：步长，可以是单个值，也可以是tuple元组
        )

        self.L2_conv_pool = torch.nn.Sequential(
            torch.nn.Conv2d(26, 52, kernel_size=3),  #输出通道数52,长度和宽度各减2
            torch.nn.BatchNorm2d(52),
            torch.nn.ReLU(inplace=True) ,
            torch.nn.MaxPool2d(kernel_size=2, stride=2) 
        )

        self.L3_fc = torch.nn.Sequential(  #全连接层
            torch.nn.Linear(52 * 5 * 5, 1024),  #由52*5*5变到1024*1，变到128*1，到10*1
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(1024, 128),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(128, 10)
        )
    
    def ForwardPropagation(self, x):
        x = self.L1_conv_pool(x)
        x = self.L2_conv_pool(x)
        x = x.view(x.size(0), -1)  # 在进入全连接层之前需要把数据拉直Flatten  
        # view()的作用相当于numpy中的reshape，重新定义矩阵的形状。
        #变行数为x.size(0)，列数随着其自动调整
        x = self.L3_fc(x)
        return x

In [4]:

def train(train_idx):
    print("train===0")
    cross_loss = 0.0
    for train_id, data in enumerate(train_process_data, 0):  #每次一个batch（64）
        Feature, Tag = data
        Feature, Tag = Feature.to(device), Tag.to(device)  # 将数据放在GPU上跑所需要的代码
        MyOptimizer.zero_grad()  #optimizer.zero_grad()意思是把梯度置零，也就是把loss关于weight的导数变成0.
        outputs = model_CNN.ForwardPropagation(Feature) 
        # print("---",target.shape)
        # print("====",outputs.shape)
        # print(target)
        LossFunction=torch.nn.CrossEntropyLoss()
        loss = LossFunction(outputs, Tag)  #对target进行onehot编码
        loss.backward()         
        #Backward函数实际上是通过传递参数(默认情况下是1x1单位张量)来计算梯度的，
        # 它通过Backward图一直到每个叶节点，每个叶节点都可以从调用的根张量追溯到叶节点。然后将计算出的梯度存储在每个叶节点的.grad中
        MyOptimizer.step()       # 所有的optimizer都实现了step()方法，这个方法会更新所有的参数。 

        #优化器就是需要根据网络反向传播的梯度信息来更新网络的参数，以起到降低loss函数计算值的作用，这也是机器学习里面最一般的方法论。
        #step这个函数使用的是参数空间(param_groups)中的grad,也就是当前参数空间对应的梯度
        #解释了为什么optimzier使用之前需要zero清零一下，因为如果不清零，那么使用的这个grad就得同上一个mini-batch有关，这不是我们需要的结果
        #我们知道optimizer更新参数空间需要基于反向梯度，因此，当调用optimizer.step()的时候应当是loss.backward()的时候
        #那么为什么optimizer.step()需要放在每一个batch训练中，而不是epoch训练中，这是因为现在的mini-batch训练模式是假定每一个训练集就
        # 只有mini-batch这样大，因此实际上可以将每一次mini-batch看做是一次训练，一次训练更新一次参数空间，因而optimizer.step()放在这里。
        cross_loss += loss.item()
        #代码中所有的loss都直接用loss表示的，结果就是每次迭代，空间占用就会增加，直到cpu或者gup爆炸。
        # 解决办法：把除了loss.backward()之外的loss调用都改成loss.item()，就可以解决。
        #可以看出是显示精度的区别，item()返回的是一个浮点型数据（更精确），所以我们在求loss或者accuracy时，一般使用item()，而不是直接取它对应的元素x[1,1]。
        
        if train_id % 100 == 0 and train_id!=0:  # 不让他每一次小的迭代就输出，而是300次小迭代再输出一次
            print("epoch %d" % train_idx,end=" ")
            print('percent: {:.00%}'.format(train_id/1000),end=" ")
            print('loss:%.7f' % (cross_loss/100),end="")
            print("")
            # print('epoch %d,{:.2f}% loss:%.7f' % (train_idx, (train_id + 1)/10, cross_loss / 100))
            cross_loss = 0.0
    torch.save(model_CNN, './modelpth/model_new_{}.pth'.format(train_idx))


def test():
    print("test===0")
    correct_sum = 0
    total_sum = 0
    with torch.no_grad():  # 下面的代码就不会再计算梯度，torch.no_grad()是一个上下文管理器，用来禁止梯度的计算，通常用来网络推断中，它可以减少计算内存的使用量。
        for data in test_process_data:
            Feature, Tag = data
            Feature, Tag = Feature.to(device), Tag.to(device)  # 将数据放在GPU上跑所需要的代码
            outputs = model_CNN.ForwardPropagation(Feature)
            Row_Max , Max_Index = torch.max(outputs.data, dim=1)  # Row_Max为每一行的最大值，Max_Index表示每一行最大值的下标
            #input是softmax函数输出的一个tensor ， dim是max函数索引的维度0/1，0是每列的最大值，1是每行的最大值
            total_sum += Tag.size(0)  #计算样本总数
            correct_sum += (Max_Index == Tag).sum().item()  
    print(type(correct_sum))
    print(type(total_sum))
    print('Accuracy（test） == %.6f %%' % (100 * correct_sum / total_sum))


if __name__ == '__main__':
    model_CNN = MyCNN_NET()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  # 将数据放在GPU上跑所需要的代码
    model_CNN.to(device)  # 将数据放在GPU上跑所需要的代码
    MyOptimizer = torch.optim.SGD(model_CNN.parameters(), lr=0.1)  
    for train_idx in range(1,19):
        train(train_idx)
        test()

train===0
epoch 1 percent: 10% loss:0.7518920
epoch 1 percent: 20% loss:0.1710116
epoch 1 percent: 30% loss:0.1355203
epoch 1 percent: 40% loss:0.0922126
epoch 1 percent: 50% loss:0.0694858
epoch 1 percent: 60% loss:0.0719873
epoch 1 percent: 70% loss:0.0593762
epoch 1 percent: 80% loss:0.0621834
epoch 1 percent: 90% loss:0.0564769
test===0
<class 'int'>
<class 'int'>
Accuracy（test） == 98.440000 %
train===0
epoch 2 percent: 10% loss:0.0401990
epoch 2 percent: 20% loss:0.0484504
epoch 2 percent: 30% loss:0.0413558
epoch 2 percent: 40% loss:0.0436657
epoch 2 percent: 50% loss:0.0354218
epoch 2 percent: 60% loss:0.0467803
epoch 2 percent: 70% loss:0.0409082
epoch 2 percent: 80% loss:0.0382148
epoch 2 percent: 90% loss:0.0398346
test===0
<class 'int'>
<class 'int'>
Accuracy（test） == 98.780000 %
train===0
epoch 3 percent: 10% loss:0.0260264
epoch 3 percent: 20% loss:0.0260472
epoch 3 percent: 30% loss:0.0280562
epoch 3 percent: 40% loss:0.0237475
epoch 3 percent: 50% loss:0.0289673
epoch 3 